In [ ]:
#? en el cmd para activar el entorno virtual
Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass
.\python\venv\Scripts\Activate.ps1


In [9]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [2]:
#terminal bey 
pip install torch torchvision torchaudio
#despues
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
#luego
pip install -U -r requirements.txt


SyntaxError: invalid syntax (4063666906.py, line 2)

In [3]:
ls

 El volumen de la unidad D es Nuevo vol
 El n£mero de serie del volumen es: 520D-5B23

 Directorio de D:\Tesis\python

29/04/2025  19:29    <DIR>          .
29/04/2025  19:29    <DIR>          ..
29/04/2025  19:34    <DIR>          .ipynb_checkpoints
13/04/2025  13:17            10.529 040425-20pm.py
12/04/2025  18:36             3.055 050425-16pm.py
22/04/2025  14:18             3.324 050425-16pm-ACTIVE.py
12/04/2025  18:36             2.923 050425-17pm.py
19/04/2025  09:01             3.039 050425-17pm-ACTIVE.py
12/04/2025  19:10             2.133 1.3-redimensionar.py
12/04/2025  18:36             2.004 1.redimensionar.py
12/04/2025  19:24            11.572 110425-21pm.py
20/04/2025  13:17            11.625 110425-21pm-ACTIVE.py
12/04/2025  18:36             3.634 1-2.redimiensinar.py
13/04/2025  16:28             2.462 130425-16pm.py
13/04/2025  18:34             2.525 130425-16pm-ACTIVE.py
22/04/2025  19:20            12.952 130425-18pm-ACTIVE.py
26/04/2025  19:08             3.083

In [4]:
YOLO_MODEL     = 'yolov5s' # puede ser yolov5m, yolov5l, yolov5x
EPOCHS         = 50
CFG            = YOLO_MODEL+'.yaml'
TRAIN_WEIGHTS  = YOLO_MODEL+'.pt'  # o pesos de otro modelo propio pre-entrenado
BATCH          = 16
IMG_SIZE       = 128  #mas cercano a 720*960
YAMLFILE       = '../dataset_yolo_split/data.yaml'
NAME           = 'dataset_yolo_split'

In [7]:
cd yolov5

D:\Tesis\yolov5


In [10]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

False


AssertionError: Torch not compiled with CUDA enabled

In [15]:
!python train.py --img {IMG_SIZE} --batch {BATCH} --epochs {EPOCHS} --data {YAMLFILE} --cfg {CFG} --weights {TRAIN_WEIGHTS} --name {NAME} --cache --exist-ok --save-period 10 --hyp data/hyps/hyp.scratch-low.yaml --rect

Traceback (most recent call last):
  File "d:\Tesis\yolov5\train.py", line 34, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


In [ ]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2

# Asegurar renderizado en notebooks
%matplotlib inline

# Cargar modelo
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path='/content/yolov5/runs/train/dataset_yolo_split/weights/last.pt')

# Cargar imagen
img_path = '/content/dataset_yolo_split/images/test/IMG_20250302_124849_box4_56.jpg'
original_img = Image.open(img_path).convert('RGB')
img_np = np.array(original_img)

# Obtener detecciones
results = model(img_path)
detections = results.pandas().xyxy[0]

# Crear copia para dibujar solo cajas
img_boxes = img_np.copy()

if not detections.empty:
    for _, row in detections.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        color = (0, 255, 0)  # Verde
        thickness = 2
        cv2.rectangle(img_boxes, (x1, y1), (x2, y2), color, thickness)

# Mostrar lado a lado
plt.figure(figsize=(18, 9))

# Imagen original
plt.subplot(1, 2, 1)
plt.imshow(original_img)
plt.title('📷 Imagen original', fontsize=14)
plt.axis('off')

# Imagen con solo cajas
plt.subplot(1, 2, 2)
plt.imshow(img_boxes)
plt.title(f'🟩 Detecciones: {len(detections)} objetos', fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

# Tabla con detalles
if not detections.empty:
    print("\n🔍 Detalles de las detecciones:")
    print("-" * 50)
    for i, (_, row) in enumerate(detections.sort_values('xmin').iterrows(), 1):
        print(f"""Detección {i}:
    Carácter: {row['name']}
    Clase: {row['class']}
    Confianza: {row['confidence']:.2%}
    Posición X: {row['xmin']:.1f}-{row['xmax']:.1f}
    Posición Y: {row['ymin']:.1f}-{row['ymax']:.1f}
        """)
    print(f"\n🔢 Secuencia detectada: {''.join(detections.sort_values('xmin')['name'].tolist())}")
else:
    print("\n⚠️ No se detectaron caracteres")